In [1]:
from datetime import datetime, timedelta
from pandas import DataFrame
from models.Payload import REGION, RegPayload, AffectedPayload
from models.Source import DataSource as SecApi
from utils.transforms import deltatime_fmt, str2date
from urllib.parse import urlencode as url
#cspell:disable

#input
region:REGION='Valparaiso'
#total regional clients
users = SecApi()
q = users.request('reg_clients',RegPayload(region=region))
clients = int(q[0]['CLIENTES'])
print(f"{"usuarios regionales":<22}",':',f"{clients:,.0f} clientes")

#data
curr = SecApi()
curr.request(source='affected_detail',payload=AffectedPayload()) 
curr_df=curr.dataframe(FECHA_INT_STR=str2date)
curr_df= curr_df[curr_df['NOMBRE_REGION']==region].drop(columns=['HORA','DIA','MES','ANHO'])
affected = curr_df['CLIENTES_AFECTADOS'].sum()

print(f'{'usuarios sin servicio':<22}',':',f'{affected:,.0f}',' clientes',f'({affected/clients:.2%})')
print(f'{'fecha reporte':<22}',':',curr.report_date.strftime("%d/%m/%Y %H:%M"))

res:DataFrame = curr_df\
    .groupby(['NOMBRE_COMUNA','NOMBRE_EMPRESA','FECHA_INT_STR'],as_index=False)['CLIENTES_AFECTADOS']\
    .sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)
res['TIEMPO_SIN_SERVICIO'] = res['FECHA_INT_STR'].apply(lambda event_date: datetime.now()-event_date+timedelta(days=0))

#cspell:disable
res:DataFrame = res.rename(columns={
    'FECHA_INT_STR':'INCIDENTE',
    'CLIENTES_AFECTADOS':'SIN_SERVICIO',
    'NOMBRE_EMPRESA':'DISTRIBUIDORA',
    'NOMBRE_COMUNA':'COMUNA'
    })

#days w/o service
res['TIEMPO_SIN_SERVICIO'] = res['INCIDENTE'].apply(deltatime_fmt)
res

usuarios regionales    : 780,563 clientes
usuarios sin servicio  : 202  clientes (0.03%)
fecha reporte          : 26/08/2024 10:03
usuarios sin servicio  : 597  clientes (0.08%)
fecha reporte          : 25/08/2024 09:59


,COMUNA,DISTRIBUIDORA,INCIDENTE,SIN_SERVICIO,TIEMPO_SIN_SERVICIO
5,Viña del Mar,CHILQUINTA,2024-02-19,155,189 days 10:03:53.070828
3,Quilpue,CHILQUINTA,2024-02-19,37,189 days 10:03:53.070828
1,Limache,CHILQUINTA,2024-08-26,7,0 days 10:03:53.070828
0,La Ligua,CGE,2024-08-26,1,0 days 10:03:53.070828
2,Petorca,CGE,2024-08-26,1,0 days 10:03:53.070828
4,Viña del Mar,CGE,2024-08-26,1,0 days 10:03:53.070828


In [2]:
res_by_city:DataFrame = curr_df.groupby(['NOMBRE_COMUNA'],as_index=False)['CLIENTES_AFECTADOS']\
    .sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)
res_by_city

,NOMBRE_COMUNA,CLIENTES_AFECTADOS
2,Viña del Mar,513
1,Quilpue,83
0,Puchuncavi,1


In [3]:
from datetime import datetime
delta = datetime.now()-datetime.strptime("19-06-2024","%d-%m-%Y")
str(delta)

'67 days, 9:59:08.616230'